# Demonstration of the QLSAs workflow, following the qiskit patterns framework

In [ ]:
# Set system path

from pathlib import Path
import sys


def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for d in (p, *p.parents):
        if (d / ".git").exists() or (d / "pyproject.toml").exists() or (d / "src").exists():
            return d
    return p  # fallback


repo_root = find_repo_root()
print(repo_root)

# This repo uses a "src layout" (the Python package lives in <repo>/src/qlsas),
# but the repo is not installed as a package. Add <repo>/src to sys.path.
src_dir = repo_root / "src"
if src_dir.exists() and str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Also add the repo root so you can import top-level modules like
# `linear_systems_problems.random_matrix_generator`.
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

In [ ]:
# Import QLSAs modules

from qlsas.qlsa.hhl import HHL
from qlsas.data_loader import StatePrep
from qlsas.transpiler import Transpiler
from qlsas.executer import Executer
from qlsas.post_processor import Post_Processor
from qlsas.solver import QuantumLinearSolver
from qlsas.refiner import Refiner
from linear_systems_problems.random_matrix_generator import generate_problem

# Import other modules
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

%config InlineBackend.figure_format = "retina"

plt.rcParams.update({
    "font.family": "serif",
    "font.size": 12,
    "axes.labelsize": 14,
    "axes.titlesize": 14,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    "figure.dpi": 150,
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
})

## Step 0: **Define** the (classical) problem

In [ ]:
prob = generate_problem(n=8, cond_number=5.0, sparsity=0.5, seed=0)
A, b = prob["A"], prob["b"]

A  = A / np.linalg.norm(b)
b = b / np.linalg.norm(b)

print(f"A: {A}")
print()
print(f"b: {b}")
print()
print(np.linalg.eigvalsh(A))

## Step 1: **Map** problem to quantum circuits and operators

In [ ]:
hhl = HHL(
    state_prep = StatePrep(method='default'),
    readout = 'measure_x', # 'measure_x' or 'swap_test'
    #swap_test_vector = np.ones(len(b)) / np.linalg.norm(np.ones(len(b))),
    num_qpe_qubits = int(math.log2(len(b))),
    t0 = 1) #2 * np.pi)

In [ ]:
hhl_circuit = hhl.build_circuit(A, b)
hhl_circuit.draw(output='mpl', fold=-1)

## Step 2: **Optimize** for target hardware

In [ ]:
service = QiskitRuntimeService(name="QLSAs")
service.backends()

In [ ]:
# Define a backend

# use a specific hardware backend
# backend = service.backend("ibm_boston")

# or use the least busy backend
# backend = service.least_busy(operational=True, min_num_qubits=hhl_circuit.num_qubits)
# print("Backend: ", backend)

# or use a simulator
# noiseless:
backend = AerSimulator()

# noisy:
# Add depolarizing error to all single qubit u1, u2, u3 gates
# noise_model = NoiseModel()
# error_prob = .05
# error = depolarizing_error(error_prob, 1)
# noise_model.add_all_qubit_quantum_error(error, ["u1", "u2", "u3"])
# backend = AerSimulator(noise_model=noise_model)

In [ ]:
# Transpile the circuit

transpiler = Transpiler(circuit=hhl_circuit, backend=backend, optimization_level=3)
transpiled_hhl_circuit = transpiler.optimize()

print(f"2q-depth:        {transpiled_hhl_circuit.depth(lambda x: x.operation.num_qubits==2)}")
print(f"2q-size:         {transpiled_hhl_circuit.size(lambda x: x.operation.num_qubits==2)}")
print(f"Operator counts: {transpiled_hhl_circuit.count_ops()}")

## Step 3: **Execute** on target hardware

In [ ]:
executer = Executer()

result = executer.run(
    transpiled_circuit = transpiled_hhl_circuit, 
    backend = backend,
    shots = 100
    )

## Step 4: **Process** result to obtain classical solution

In [ ]:
processor = Post_Processor()
solution, success_rate, residual = processor.process_tomography(result, A, b)

## Wrapping steps 1-4 together in a solver:

In [ ]:
hhl_solver = QuantumLinearSolver(
    qlsa = hhl,
    backend = backend,
    target_successful_shots = 1000,
    shots_per_batch = 5000,
    optimization_level = 3,
    executer = executer,
    post_processor = processor,)
    #mode = "session")

## Integrate **Iterative Refinement** to improve accuracy

In [ ]:
refiner = Refiner(A = A, b = b, solver = hhl_solver)
refined_solution = refiner.refine(precision = 1e-9, max_iter = 3, plot=True)

## Experiments

In [ ]:
t0_values = np.linspace(0, 2 * np.pi, 30)
success_rates = []
residual_errors = []

prob = generate_problem(n=8, cond_number=10.0, sparsity=0.5, seed=0)
A, b = prob["A"], prob["b"]
A = A / np.linalg.norm(b)
b = b / np.linalg.norm(b)

for t0 in t0_values:
    if np.isclose(t0, 0.0):
        # t0=0 causes divide-by-zero in phase-to-eigenvalue mapping
        success_rates.append(np.nan)
        residual_errors.append(np.nan)
        continue

    hhl = HHL(
        state_prep=StatePrep(method="default"),
        readout="measure_x",
        num_qpe_qubits=int(math.log2(len(b))),
        t0=t0,
    )

    hhl_circuit = hhl.build_circuit(A, b)
    transpiler = Transpiler(circuit=hhl_circuit, backend=backend, optimization_level=3)
    transpiled_hhl_circuit = transpiler.optimize()

    result = executer.run(
        transpiled_circuit=transpiled_hhl_circuit,
        backend=backend,
        shots=100,
    )

    solution, success_rate, residual = processor.process_tomography(result, A, b)
    success_rates.append(success_rate)

    residual_errors.append(residual)

In [ ]:
fig, ax1 = plt.subplots(figsize=(6, 4))
ax1.plot(t0_values, success_rates, marker="o", color="C0", label="Success rate")
ax1.set_xlabel("t0")
ax1.set_ylabel("Success rate", color="C0")
ax1.set_xlim(0, 2 * np.pi)
#ax1.set_ylim(0, 1)
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(t0_values, (residual_errors), marker="s", color="C1", label="Residual error")
ax2.set_ylabel("Residual error", color="C1")
ax2.set_ylim(bottom=0)

lines = ax1.get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels)

ax1.set_title("t0 vs success rate and residual error")
plt.tight_layout()
plt.show()

In [ ]:
iterations = 15 #15          # number of IR iterations to plot (0..iterations-1)
n = 64 #64
max_qpe_qubits = 8 #8      # sweep num_qpe_qubits = 1..max_qpe_qubits

# rows: qpe qubits, cols: IR iteration
qpe_residuals = np.full((max_qpe_qubits, iterations), np.nan, dtype=float)
total_circuit_depth = np.full((max_qpe_qubits, iterations), np.nan, dtype=float)

prob = generate_problem(n=n, cond_number=5.0, sparsity=0.5, seed=0)
A, b = prob["A"], prob["b"]
A = A / np.linalg.norm(b)
b = b / np.linalg.norm(b)

for qpe_qubits in range(1, max_qpe_qubits + 1):
    print(f"================num_qpe_qubits: {qpe_qubits}=================")
    solver = QuantumLinearSolver(
        qlsa=HHL(
            state_prep=StatePrep(method="default"),
            readout="measure_x",
            num_qpe_qubits=qpe_qubits,
            t0=2 * np.pi,
        ),
        backend=AerSimulator(),
        target_successful_shots=1000,
        shots_per_batch=5000,
        optimization_level=3,
        executer=executer,
        post_processor=processor,
    )

    refiner = Refiner(A=A, b=b, solver=solver)
    refined_solution = refiner.refine(precision=1e-20, max_iter=iterations - 1, plot=False)

    r = np.asarray(refined_solution["residuals"], dtype=float)
    qpe_residuals[qpe_qubits - 1, : len(r)] = r

    circuits = refined_solution["transpiled_circuits"]
    depths = np.array([tc.depth() for tc in circuits], dtype=float)
    cumulative_depth = np.cumsum(depths)
    total_circuit_depth[qpe_qubits - 1, : len(cumulative_depth)] = cumulative_depth

# heatmap of log10 residuals
log_qpe_residuals = np.log10(np.maximum(qpe_residuals, 1e-16))
log_total_depth = np.log10(np.maximum(total_circuit_depth, 1))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
im = ax.imshow(
    log_qpe_residuals,
    origin="lower",
    aspect="auto",
    cmap="viridis",
    interpolation="bilinear",
)

ax.set_xlabel("Iterative refinement iteration")
ax.set_ylabel("Number of QPE qubits")

ax.set_xticks(np.arange(iterations))
ax.set_yticks(np.arange(max_qpe_qubits))
ax.set_yticklabels(np.arange(1, max_qpe_qubits + 1))

cbar = fig.colorbar(im, ax=ax, pad=0.02)
cbar.set_label(r"$\log_{10}\!\left(\|Ax - b\|_2\right)$")
cbar.ax.tick_params(labelsize=11)

X, Y = np.meshgrid(np.arange(iterations), np.arange(max_qpe_qubits))
cs = ax.contour(X, Y, total_circuit_depth, levels=[8000, 16000, 32000, 64000, 128000],
                colors="white", linestyles="--", linewidths=0.9)
labels = ax.clabel(cs, fmt=lambda v: f"{int(v):,}", fontsize=9, colors="white")
outline = [pe.withStroke(linewidth=2, foreground="black")]
for lbl in labels:
    lbl.set_path_effects(outline)

ax.plot([], [], ls="--", color="gray", label="Cumulative circuit depth")
ax.legend(loc="upper right", facecolor="white", framealpha=1.0, edgecolor="gray")

plt.tight_layout()
# plt.savefig("qpe_vs_ir_residuals.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
im = ax.imshow(
    total_circuit_depth,
    origin="lower",
    aspect="auto",
    cmap="magma",
    interpolation="nearest",
)

ax.set_xlabel("Iterative refinement iteration")
ax.set_ylabel("Number of QPE qubits")

ax.set_xticks(np.arange(iterations))
ax.set_yticks(np.arange(max_qpe_qubits))
ax.set_yticklabels(np.arange(1, max_qpe_qubits + 1))

cbar = fig.colorbar(im, ax=ax)
cbar.set_label(r"$\mathrm{cumulative\ circuit\ depth}$")

plt.tight_layout()
plt.show()

In [ ]:
iterations = 11 #11          # number of IR iterations to plot (0..iterations-1)
n = 64 #64
shots = np.linspace(1, 1000, 5, dtype=int)

# rows: shot counts, cols: IR iteration
shot_residuals = np.full((len(shots), iterations), np.nan, dtype=float)

prob = generate_problem(n=n, cond_number=5.0, sparsity=0.5, seed=0)
A, b = prob["A"], prob["b"]
A = A / np.linalg.norm(b)
b = b / np.linalg.norm(b)

num_qpe_qubits = int(math.log2(len(b)))

for i, shot_count in enumerate(shots):
    print(f"================shots: {shot_count}=================")
    solver = QuantumLinearSolver(
        qlsa=HHL(
            state_prep=StatePrep(method="default"),
            readout="measure_x",
            num_qpe_qubits=num_qpe_qubits,
            t0=2 * np.pi,
        ),
        backend=AerSimulator(),
        target_successful_shots=shot_count,
        shots_per_batch=5 * shot_count,
        optimization_level=3,
        executer=executer,
        post_processor=processor,
    )

    refiner = Refiner(A=A, b=b, solver=solver)
    refined_solution = refiner.refine(precision=1e-20, max_iter=iterations - 1, plot=False)

    r = np.asarray(refined_solution["residuals"], dtype=float)
    shot_residuals[i, : len(r)] = r

# heatmap of log10 residuals
log_shot_residuals = np.log10(np.maximum(shot_residuals, 1e-16))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
im = ax.imshow(
    log_shot_residuals,
    origin="lower",
    aspect="auto",
    cmap="viridis",
    interpolation="nearest",
)

ax.set_xlabel("Iterative refinement iteration")
ax.set_ylabel("Shots per iteration")

ax.set_xticks(np.arange(iterations))
ax.set_yticks(np.arange(len(shots)))
ax.set_yticklabels(shots)

cbar = fig.colorbar(im, ax=ax, pad=0.02)
cbar.set_label(r"$\log_{10}\!\left(\|Ax - b\|_2\right)$")
cbar.ax.tick_params(labelsize=11)

X, Y = np.meshgrid(np.arange(iterations), np.arange(len(shots)))
cumulative_shots = shots[:, None] * (X + 1)
levels = [500, 1000, 2000, 4000, 8000]
cs = ax.contour(X, Y, cumulative_shots, levels=levels,
                colors="white", linestyles="--", linewidths=0.9)
labels = ax.clabel(cs, fmt=lambda v: f"{int(v):,}", fontsize=9, colors="white")
outline = [pe.withStroke(linewidth=2, foreground="black")]
for lbl in labels:
    lbl.set_path_effects(outline)

ax.plot([], [], ls="--", color="gray", label="Cumulative shots")
ax.legend(loc="upper right", facecolor="white", framealpha=1.0, edgecolor="gray")

plt.tight_layout()
# plt.savefig("shots_vs_ir_residuals.pdf")
plt.show()

In [ ]:
# n = 64
# max_qpe_qubits = 6
# shot_values = np.linspace(1, 200, 21, dtype=int)

# single_solve_residuals = np.full((max_qpe_qubits, len(shot_values)), np.nan, dtype=float)

# prob = generate_problem(n=n, cond_number=5.0, sparsity=0.5, seed=0)
# A, b = prob["A"], prob["b"]
# A = A / np.linalg.norm(b)
# b = b / np.linalg.norm(b)
# x_classical = np.linalg.solve(A, b)

# for qi, qpe_qubits in enumerate(range(1, max_qpe_qubits + 1)):
#     for si, shot_count in enumerate(shot_values):
#         print(f"qpe={qpe_qubits}, shots={shot_count}")
#         solver = QuantumLinearSolver(
#             qlsa=HHL(
#                 state_prep=StatePrep(method="default"),
#                 readout="measure_x",
#                 num_qpe_qubits=qpe_qubits,
#                 t0=2 * np.pi,
#             ),
#             backend=AerSimulator(),
#             target_successful_shots=int(shot_count),
#             shots_per_batch=5 * int(shot_count),
#             optimization_level=3,
#             executer=executer,
#             post_processor=processor,
#         )
#         x_q = solver.solve(A, b, verbose=False)
#         alpha = np.dot(A @ x_q, b) / np.dot(A @ x_q, A @ x_q)
#         x_scaled = alpha * x_q
#         single_solve_residuals[qi, si] = np.linalg.norm(b - A @ x_scaled)

# log_single_residuals = np.log10(np.maximum(single_solve_residuals, 1e-16))

In [ ]:
# fig, ax = plt.subplots(figsize=(7, 4))
# im = ax.imshow(
#     single_solve_residuals,
#     origin="lower",
#     aspect="auto",
#     cmap="viridis",
#     interpolation="nearest",
# )

# ax.set_xlabel("Shots")
# ax.set_ylabel("Number of QPE qubits")

# ax.set_xticks(np.arange(len(shot_values)))
# ax.set_xticklabels(shot_values, rotation=45, ha="right")
# ax.set_yticks(np.arange(max_qpe_qubits))
# ax.set_yticklabels(np.arange(1, max_qpe_qubits + 1))

# cbar = fig.colorbar(im, ax=ax, pad=0.02)
# cbar.set_label(r"$\log_{10}\!\left(\|Ax - b\|_2\right)$")
# cbar.ax.tick_params(labelsize=11)

# plt.tight_layout()
# # plt.savefig("qpe_vs_shots_residuals.pdf")
# plt.show()

In [ ]:
# from matplotlib.colors import ListedColormap

# converges = single_solve_residuals < 1

# cmap_rg = ListedColormap(["#d62728", "#2ca02c"])

# fig, ax = plt.subplots(figsize=(7, 4))
# im = ax.imshow(
#     converges.astype(int),
#     origin="lower",
#     aspect="auto",
#     cmap=cmap_rg,
#     interpolation="nearest",
#     vmin=0,
#     vmax=1,
# )

# ax.set_xlabel("Shots")
# ax.set_ylabel("Number of QPE qubits")

# ax.set_xticks(np.arange(len(shot_values)))
# ax.set_xticklabels(shot_values, rotation=45, ha="right")
# ax.set_yticks(np.arange(max_qpe_qubits))
# ax.set_yticklabels(np.arange(1, max_qpe_qubits + 1))

# from matplotlib.patches import Patch
# legend_handles = [
#     Patch(facecolor="#2ca02c", edgecolor="gray", label=r"$\|r\| < 1$ (IR will converge)"),
#     Patch(facecolor="#d62728", edgecolor="gray", label=r"$\|r\| \geq 1$ (IR will not converge)"),
# ]
# ax.legend(handles=legend_handles, loc="upper left", facecolor="white", framealpha=1.0, edgecolor="gray")

# plt.tight_layout()
# # plt.savefig("qpe_vs_shots_convergence.pdf")
# plt.show()